# Chain

**Chain**(체인)은 여러 컴포넌트(요소)를 정해진 순서대로 연결하여 **복잡한 AI 작업을 단계별로 자동화**할 수 있도록 돕는 구조이다.

- 각 컴포넌트는 입력을 받아 특정 처리를 수행한 후 다음 단계로 결과를 전달한다.
- 복잡한 작업을 여러 개의 단순한 단계로 나누고, 각 단계를 순차적으로 실행함으로써 전체 작업을 체계적으로 구성할 수 있다.

## 기본 개념

- 체인은 하나의 LLM 호출에 그치지 않고 **여러 LLM 호출이나 도구 실행을 순차적으로 연결**할 수 있다.
- 예를 들어, 사용자의 질문 → 검색 → 요약 → 응답 생성 같은 일련의 작업을 체인으로 구성할 수 있다.
- 체인을 사용하면 코드의 재사용성과 유지 보수성이 향상된다.

## LangChain에서의 Chain 구성 방식

LangChain은 다음 두 가지 방식을 통해 체인을 구성할 수 있다.

### 1. Off-the-shelf Chains 방식 (클래식 방식)

- LangChain에서 제공하는 **미리 정의된 Chain 클래스**(예: `LLMChain`, `SequentialChain`, `SimpleSequentialChain`)를 활용하는 방식이다.
- 이 방식은 LangChain의 **초기 구조**이며, 대부분의 클래스는 현재 **더 이상 사용되지 않음(deprecated)** 상태이다.

> 현재 LangChain에서는 이 방식을 권장하지 않는다.

### 2. LCEL (LangChain Expression Language) 방식

- 체인을 함수형 방식으로 선언할 수 있는 **표현식 기반의 체인 구성 언어**이다.
- LCEL 방식은 간결하고 선언적인 문법을 제공하여 **직관적이고 확장성 있는 체인 구성**이 가능하다.
- `Runnable`이라는 공통 인터페이스를 기반으로 다양한 요소를 조합하여 체인을 구성한다.
- 체인의 각 구성 요소는 `invoke()` 메서드로 실행된다.

In [2]:
# off-the-shelf 방식

from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

prompt_template = PromptTemplate(template="{item}에 어울리는 이름 {count}개 만들어 주세요.")

model = ChatOpenAI(model_name="gpt-4o-mini")
parser = StrOutputParser()

In [4]:
from langchain import LLMChain

chain = LLMChain(
    prompt = prompt_template,
    llm =model,
    output_parser = parser
)

response = chain.invoke({"item":"가방","count":5})

C:\Users\tjddm\AppData\Local\Temp\ipykernel_7012\1148074225.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


In [5]:
response

{'item': '가방',
 'count': 5,
 'text': '물론이죠! 가방에 어울리는 이름 5개를 제안합니다.\n\n1. **루비백** - 고급스럽고 세련된 느낌을 주는 이름\n2. **테라코타** - 따뜻한 색감을 연상시키는 자연 친화적인 이름\n3. **모멘트백** - 특별한 순간을 담아내는 의미의 이름\n4. **에코백스** - 환경을 생각하는 친환경 가방에 적합한 이름\n5. **시크릿 포켓** - 숨겨진 공간이 많은 가방에 어울리는 미스터리한 이름\n\n이름들이 마음에 드시길 바랍니다!'}

# [LCEL](https://python.langchain.com/docs/how_to/#langchain-expression-language-lcel) (LangChain Expression Language)
- LCEL은 LangChain의 핵심 기능인 체인(Chain)을 더욱 간결하고 유연하게 구성할 수 있도록 고안된 **선언형 체인(chain) 구성 언어**이다.
- 파이프 연산자 `|`를 사용해 선언적 방법으로 여러 작업을 연결한다.
- 체인을 구성하는 각 요소는 `Runnable` 타입으로, 체인 내에서 실행 가능한 단위이다.
- 각 단계는 invoke() 메서드를 통해 실행되며, 앞 단계의 출력이 다음 단계의 입력으로 자동 전달된다.
    - [Runnable 컴포넌트별 입출력 타입](https://python.langchain.com/docs/concepts/runnables/#input-and-output-types)
    - 각 컴포넌트의 input과 output 타입에 맞춰 값이 전달되도록 한다.
- https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel

## [Runnable](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.Runnable.html)
- LangChain의 Runnable은 실행 가능한 작업 단위를 캡슐화한 개념으로, 데이터 흐름의 각 단계를 정의하고 **체인(chain) 에 포함 되어**  복잡한 작업의 각 단계를 수행 한다.
- Chain을 구성하는 class들은 Runnable의 상속 받아 구현한다.
- **Prompt Template클래스**, **Chat 모델, LLM 모델 클래스**, **Output Parser 클래스** 등 다양한 컴포넌트가 Runnable을 상속받아 구현된다.

### 주요 특징
- 작업 단위의 캡슐화:
    - Runnable은 특정 작업(예: 프롬프트 생성, LLM 호출, 출력 파싱 등)을 수행하는 독립적인 컴포넌트이다.
    - 각 컴포넌트는 독립적으로 테스트 및 재사용이 가능하며, 조합하여 복잡한 체인을 구성할 수 있다.
- 체인 연결 및 작업 흐름 관리:
    - Runnable은 체인(chain, 일련의 연결된 작업 흐름)을 구성하는 기본 단위로 사용된다.
    - LangChain Expression Language(LCEL)를 사용하면 | 연산자를 통해 여러 Runnable을 쉽게 연결할 수 있다.
    - 입력과 출력의 형식을 일관되게 유지하여 각 단계가 자연스럽게 연결된다.
- 모듈화 및 디버깅 용이성:
    - 각 단계가 명확히 분리되어 문제 발생 시 어느 단계에서 오류가 발생했는지 쉽게 확인할 수 있다.
    - 복잡한 작업을 작은 단위로 나누어 체계적으로 관리할 수 있다.
      
### Runnable의 표준 메소드
- 모든 Runnable이 구현하는 공통 메소드
    - `invoke()`: 단일 입력을 처리하여 결과를 반환.
    - `batch()`: 여러 입력 데이터들을 한 번에 처리.
    - `stream()`: 입력에 대해 스트리밍 방식으로 응답을 반환.
    - `ainvoke()`: 비동기 방식으로 입력을 처리하여 결과를 반환.

### Runnable의 주요 구현체(하위 클래스)

- `RunnableSequence`
    - 여러 `Runnable`을 순차적으로 연결하여 실행하는 구성이다.
    - 각 단계의 출력이 다음 단계의 입력으로 전달된다.
    - LCEL을 사용하여 체인을 구성할 경우 자동으로 `RunnableSequence`로 변환된다.
-  `RunnablePassThrough`
    - 입력 데이터를 가공하지 않고 그대로 다음 단계로 전달하는 `Runnable`이다.
    - 선택적으로 미리 정의된 키-값 쌍을 함께 전달할 수 있다.

- `RunnableParallel`
    - 여러 `Runnable`을 병렬로 실행한 후, 결과를 결합하여 다음 단계로 전달한다.
    - 병렬 처리를 통해 처리 속도를 개선할 수 있다.

- `RunnableLambda`
    - 일반 함수 또는 `lambda` 함수를 `Runnable`로 변환하여 체인에 포함할 수 있다.
    - 사용자 정의 함수로 동작을 확장할 때 유용하다.

#### Runnable 예제

In [9]:
from langchain_core.runnables import Runnable # 모든 Runnable의 최상위

# 사용자 정의 Runnable
class MyRunnable(Runnable):

    def invoke(self, input_data:str, config:dict=None):
        # invoke() : 구현하는 Runnable이 해야하는 작업을 구현하는 메소드.
        # input_data : 입력값
        # config : 필요한 설정 값들.
        if config is not None and config.get("lang") == "en":
            return f"Explain {input_data} in one sentence."

        return f"{input_data}에 대해서 한 문장으로 설명해줘."

In [10]:
my_runnabel = MyRunnable()
my_runnabel.invoke("사과")
my_runnabel.invoke("컴퓨터")
my_runnabel.invoke("Apple", {"lang":"en"})

'Explain Apple in one sentence.'

In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

my_runnabel = MyRunnable()
model = ChatOpenAI(model_name="gpt-4o-mini")
parser = StrOutputParser()

prompt = my_runnabel.invoke("apple", {"lang":"en"})

response = model.invoke(prompt)

print(response)

content='An apple is a round fruit produced by the apple tree, typically characterized by its sweet, crisp flesh and a smooth skin, often available in various colors such as red, green, and yellow.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 13, 'total_tokens': 52, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BgjcJD04Zhm3RbSvyj30dr7YmRxh1', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--4304588d-1506-4d9e-aef2-4f2b986dd166-0' usage_metadata={'input_tokens': 13, 'output_tokens': 39, 'total_tokens': 52, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [15]:
chain = my_runnabel | model | parser

res = chain.invoke("과일 배")
print(res)

과일 배는 달콤하고 아삭한 식감이 특징인 과일로, 주로 수확 후 생으로 소비되거나 다양한 요리에 활용됩니다.


In [25]:
# 기본 체인 구성 : prompt_template -> model -> output parser

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser, StrOutputParser

# role : system, user/human, ai/assistant
#        system : 채팅 전체에 적용되는 공통 지침을 지정하는 role

prompt_template = ChatPromptTemplate(
    messages = [
        ("system", "당신은 오랜 경력의 한국 관광 가이드입니다. 여행객들에게 설명하듯이 답변을 해주세요."),
        ("human", "{query}")
    ]
)

model = ChatOpenAI(model_name = "gpt-4o-mini")

guide_chain = prompt_template | model | StrOutputParser()

print(type(guide_chain)) # RunnableSequence : Runnable 타입 -> chain도 다른 chain에 구성요소로 포함 될수있다.



<class 'langchain_core.runnables.base.RunnableSequence'>


In [26]:
query = "서울에서 꼭 가봐야되는 여행지를 세 곳만 알려줘."
response = guide_chain.invoke({"query":query})

In [28]:
while True:
    query = input("질문:")
    if query == "!quit":
        break
    resp = guide_chain.invoke({"query":query})
    print("User:", query)
    print("AI:", resp)
    print("-"*50)

User: 
AI: 안녕하세요! 한국의 매력적인 관광지와 문화에 대해 궁금한 점이 있으신가요? 전통과 현대가 공존하는 아름다운 나라, 한국에 대해 자세히 설명해 드리겠습니다. 예를 들어, 서울에서는 경복궁과 같은 역사적인 장소를 방문하실 수 있고, 현대적인 매력이 넘치는 명동이나 동대문 디자인 플라자도 즐기실 수 있습니다. 또는 부산의 해운대와 같은 멋진 해변에서 휴식을 취해보세요. 어떤 주제에 대해 이야기해 볼까요?
--------------------------------------------------


#### RunnableLambda 예제

In [32]:
from langchain_core.runnables import RunnableLambda

lambda input_data : f"{input_data}를 한 문장으로 설명해줘."

# RunnableLamda(함수) -> 함수를 실행하는 Runnable을 생성.
my_runnabel2 = RunnableLambda(lambda input_data : f"{input_data}를 한 문장으로 설명해줘.")

my_runnabel2.invoke("LLM")

'LLM를 한 문장으로 설명해줘.'

In [ ]:
def sum(nums):
    return nums[0] + nums[1]

my_runnabel2 = RunnableLambda(sum)

my_runnabel2.invoke([1,2])

# invoke(입력데이터 : str | dict, 설정정보 : dict)
# 입력데이터가 여러개일 경우 dict등의 자료구조를 이용해서 받음.

3

#### RunnablePassThrough 예제

In [36]:
# 앞 Runnable이 처리한 결과를 다음 Runnable에 그대로 전달.
from langchain_core.runnables import RunnablePassthrough

RunnablePassthrough().invoke("안녕하세요")
RunnablePassthrough().invoke({"key":"value"})

{'key': 'value'}

In [37]:
# 앞 Runnable이 처리한 결과에 Item을 추가해서 다음 Runnable에 그대로 전달.
# -> 입력받은 dict에 item을 추가.

#RunnablePassthrough.assign(key=Runnable, key=Runnable, key=Runnable, ...)
# -> 받은 dict에 "key1" : Runnable 반환값, "key2" : Runnable 반환값,  ... 추가해서 다음으로 전달.

address_runnable = RunnableLambda(lambda x : "서울시 금천구")
phone_runnable = RunnableLambda(lambda x : "010-1111-2222")

RunnablePassthrough.assign(address = address_runnable, phone = phone_runnable).invoke({"name":"홍길동"})

{'name': '홍길동', 'address': '서울시 금천구', 'phone': '010-1111-2222'}

#### RunnableSequence 예제

In [ ]:
from langchain_core.runnables import RunnableSequence

run1 = RunnableLambda(lambda x : x+1)
run2 = RunnableLambda(lambda x : x*2)

chain = run1 | run2

chain.invoke(30)

62

In [40]:
chain2 = RunnableSequence(run1,run2)

chain2.invoke(30)

62

#### RunnableParallel 예제

In [45]:
from langchain_core.runnables import RunnableParallel

run1 = RunnableLambda(lambda x : x+1)
run2 = RunnableLambda(lambda x : x*2)
run3 = RunnableLambda(lambda x : x//3)

chain = RunnableParallel(
    {
        "result1" : run1,
        "result2" : run2,
        "result3" : run3,
        "result4" : RunnablePassthrough()
    }
)
# 각 Runnable들을 각각 실행하고 그 결과를 key에 할당한 Dictionary에 반환.

chain.invoke(6)


{'result1': 7, 'result2': 12, 'result3': 2, 'result4': 6}

In [63]:
a = {"food":"pasta"}

print(a.values())

dict_values(['pasta'])


#### LCEL Chain 예제

In [71]:
# 음식 이름을 받아서 레시피를 영어로 출력하는 chain을 구성
# prompt template -> model -> output parser

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from textwrap import dedent

prompt_template = PromptTemplate(
    template = dedent("""
    # instruction
    당신은 숙련된 요리 연구가입니다 요청된 음식의 레시피를 작성해 주세요.
                      
    # Input data
    음식이름 : {food}
                      
    # Output indicator
    - 레시피는 영어로 작성해 주세요.                
    """)
)

model = ChatOpenAI(model_name="gpt-4o-mini")

food_chain = prompt_template | model | StrOutputParser()

In [74]:
response = food_chain.invoke({"food":"pasta"})

In [75]:
print(response)

## Pasta Recipe

### Ingredients:
- 300g pasta (spaghetti, penne, or your choice)
- 4 liters water
- 2 tablespoons salt
- 2 tablespoons olive oil
- 3 cloves garlic, minced
- 400g canned tomatoes (crushed or diced)
- 1 teaspoon dried oregano
- 1 teaspoon dried basil (or a handful of fresh basil)
- Salt and pepper to taste
- Grated Parmesan cheese (for serving)
- Fresh basil leaves (for garnish, optional)

### Instructions:

1. **Cook the Pasta:**
   - In a large pot, bring 4 liters of water to a rolling boil.
   - Add 2 tablespoons of salt to the boiling water.
   - Add the pasta to the pot and cook according to the package directions until al dente (usually about 8-12 minutes).
   - Once cooked, reserve about 1 cup of pasta water, then drain the pasta and set aside.

2. **Prepare the Sauce:**
   - In a large skillet, heat 2 tablespoons of olive oil over medium heat.
   - Add the minced garlic and sauté for about 1 minute, or until fragrant (be careful not to burn it).
   - Add the cann

In [85]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from textwrap import dedent

prompt_template_trans = PromptTemplate(
    template = dedent("""
    # instruction
    당신은 번역 전문가 입니다. 
    문장과 입력받아 문장을 해당 {language}로 번역해주세요.
                      
    # Input data
    문장 : {content}
                      
    # Output indicator
    - 번역한 내용만 출력해 주세요.      
    """)
)

translate_chain = prompt_template_trans | model | StrOutputParser()


In [83]:
response_kor = translate_chain.invoke({"content":response,"language":"한국어"})
# 안녕하세요 -> 독일어로 번역한 응답

In [84]:
print(response_kor)

## 파스타 레시피

### 재료:
- 파스타 300g (스파게티, 펜네 또는 원하는 종류)
- 물 4리터
- 소금 2큰술
- 올리브 오일 2큰술
- 다진 마늘 3쪽
- 통조림 토마토 400g (부서진 것 또는 깍둑썰기 한 것)
- 마른 오레가노 1작은술
- 마른 바질 1작은술 (또는 신선한 바질 한 줌)
- 소금과 후추는 기호에 맞게
- 파르메산 치즈 (제공용)
- 신선한 바질 잎 (장식용, 선택 사항)

### 조리법:

1. **파스타 조리하기:**
   - 큰 냄비에 물 4리터를 끓입니다.
   - 끓는 물에 소금 2큰술을 넣습니다.
   - 파스타를 냄비에 넣고 패키지 지침에 따라 알 덴테(보통 8-12분 정도)로 조리합니다.
   - 조리가 끝나면 약 1컵의 파스타 물을 남겨두고, 나머지는 체에 밭쳐 놓습니다.

2. **소스 준비하기:**
   - 큰 팬에 올리브 오일 2큰술을 중불에서 가열합니다.
   - 다진 마늘을 넣고 약 1분간 볶아 향이 나도록 합니다(타지 않도록 주의).
   - 팬에 통조림 토마토와 마른 오레가노, 마른 바질을 넣고 섞습니다.
   - 소스를 약 10-15분 동안 끓여 농도가 진해지도록 합니다. 가끔 저어줍니다.
   - 기호에 맞게 소금과 후추로 간을 합니다.

3. **파스타와 소스 결합하기:**
   - 소스가 진해지면 drained 파스타를 팬에 넣습니다. 소스와 고르게 섞이도록 버무립니다.
   - 소스가 너무 뻑뻑하게 느껴지면 남겨둔 파스타 물을 조금씩 추가해 원하는 농도로 조절합니다.

4. **서빙하기:**
   - 파스타를 서빙용 그릇에 나눕니다.
   - 기호에 따라 간 파르메산 치즈와 신선한 바질 잎으로 토핑합니다.
   - 맛있는 홈메이드 파스타를 즐기세요!

### 팁:
- 이 레시피는 채소(시금치나 피망)나 단백질(구운 치킨이나 새우)을 추가하여 맞춤형으로 만들 수 있습니다.
- 매콤한 맛을 원하신다면 마늘을 볶을 때 붉은 고추 조각을 추가하세요.


------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# 음식 이름을 받아서 레시피를 영어로 출력하는 chain을 구성
# prompt template -> model -> output parser

from langchain_core.runnables import Runnable

# 사용자 정의 Runnable
class MyRunnable(Runnable):

    def invoke(self, input_data:dict, config:dict=None):
        
        return f"Explain {input_data.values()} how to make this food in English."

food_chain = MyRunnable() | model | parser



In [68]:
response = food_chain.invoke({"food":"햄버거"}) # 음식이름은 아무거나 넣고 결과값은 영어로 나오게

In [69]:
print(response)

To make a hamburger, you'll need the following ingredients and follow these steps:

### Ingredients:
- Ground beef (80/20 lean-to-fat ratio is ideal)
- Salt and pepper
- Burger buns
- Cheese (optional, e.g., cheddar, American)
- Toppings (lettuce, tomato, onion, pickles)
- Condiments (ketchup, mustard, mayonnaise)

### Instructions:

1. **Prepare the Patties:**
   - Take about ¼ pound (113g) of ground beef for each patty. 
   - Gently shape the meat into a round patty about ¾ inch thick. Try not to overwork the meat to keep it tender.
   - Use your thumb to create a slight indent in the center of the patty; this helps it cook evenly.
   - Season both sides with salt and pepper.

2. **Cook the Patties:**
   - Preheat a grill or skillet over medium-high heat. If using a skillet, add a small amount of oil to prevent sticking.
   - Place the patties on the grill or skillet and cook for about 3-4 minutes on one side.
   - Flip the patties and cook for another 3-4 minutes for medium-rare. Ad

## Chain과 Chain간의 연결

In [102]:
# food_chain + translate_chain
# food_chain : 변수 - food
# translate_chain - 변수 - content, language

# RunnableParallel({"key":Runnable,"key":Runnable})
# LCEL에서 RunnableParallel -> {"key":Runnable,"key":Runnable} |

chain = {"content":food_chain,
          "language":RunnableLambda(lambda x: x["language"])} | translate_chain

In [103]:
res = chain.invoke({"food":"파스타","language":"한국어"})
# food -> food_chain, language -> translate_chain
# food_chain 최종결과(레시피) -> {"content":"레시피"} -> translate_chain

In [104]:
print(res)

## 파스타 레시피

### 재료:
- 스파게티 200g (또는 선호하는 파스타)
- 올리브 오일 2큰술
- 다진 마늘 3쪽
- 붉은 고추 조각 1/2 티스푼 (선택 사항)
- 다진 토마토 400g (통조림)
- 소금과 후추, 기호에 따라
- 신선한 바질 잎, 장식용
- 파마산 치즈, 제공용

### 조리 방법:

1. **파스타 삶기:**
   - 큰 냄비에 소금물 끓이기. 스파게티를 넣고 포장지에 안내된 대로 알 덴테가 될 때까지 요리합니다. 약 1컵의 파스타 물을 남기고, 나머지는 물기를 빼고 따로 둡니다.

2. **소스 준비:**
   - 큰 팬이나 소스팬에 중불로 올리브 오일을 가열합니다. 다진 마늘과 붉은 고추 조각을 넣고 향이 날 때까지 약 1-2분간 볶습니다.

3. **토마토 추가:**
   - 다진 토마토(국물이 포함된)를 붓고 끓입니다. 소금과 후추로 간을 합니다. 약 10-15분 동안 조리하여 맛이 섞이고 소스가 약간 걸쭉해질 때까지 끓입니다.

4. **파스타와 소스 결합:**
   - 삶은 스파게티를 소스에 추가합니다. 섞이도록 버무리고, 원하는 소스 농도가 될 때까지 남겨두었던 파스타 물을 조금씩 추가합니다.

5. **서빙:**
   - 파스타를 접시에 담고 신선한 바질 잎으로 장식합니다. 원한다면 갈은 파마산 치즈와 추가 후추를 뿌립니다.

6. **맛있게 드세요!**

### 팁:
- 이 레시피는 시금치, 피망, 호박과 같은 채소나 구운 닭고기나 새우와 같은 단백질을 추가하여 맞춤 설정할 수 있습니다.
- 더 크리미한 소스를 원하면 요리 마지막에 생크림이나 크림치즈를 한 덩어리 넣어 저어주세요.

맛있는 홈메이드 파스타를 즐기세요!


# 사용자 함수를 Chain에 적용하기

## 사용자 함수를 Runnable로 정의 (RunnableLambda)
- 임의의 함수를 Runnable로 정의 할 수있다.
  - chain에 포함할 기능을 함수로 정의할 때 주로 사용. 
- `RunnableLambda(함수)` 사용
  - 함수는 invoke() 메소드를 통해 입력받은 값을 받을 **한개의 파라미터**를 선언해야 한다.

## 사용자 함수를 Chain으로 정의
- Chain 을 구성하는 작업 사이에 추가 작업이 필요할 경우, 중간 결과를 모두 사용해야 하는 경우 함수로 구현한다.
- `@chain` 데코레이터를 사용해 함수에 선언한다.

### Runnable 에 사용할 **사용자 정의 함수** 구문
- 이전 Chain의 출력을 입력 받는 **파라미터를 한개** 선언한다. (첫번째 파라미터)
- `invoke()`로 호출 할때 전달 하는 추가 설정을 입력받는 파라미터를 선언한다.(두번째 파라미터 - Optional)
  - RunnableConfig 타입의 값을 받는데 Dictionary 형식으로 `{"configuable": {"설정이름":"설정값"}}` 형식으로 받는다.
- 만약 함수가 여러개의 인자를 받는 경우 단일 입력을 받아들이고 이를 여러 인수로 풀어내는 래퍼 함수를 작성하여 Runnable로 만든다.
  ```python
  def plus(num1, num2):
      ...

  def wrapper_plus(nums:dict|list):
      return plus(nums['num1'], nums['num2'])
  ```

# Cache

- 응답 결과를 저장해서 같은 질문이 들어오면 LLM에 요청하지 않고 저장된 결과를 보여주도록 한다.
    - 처리속도와 비용을 절감할 수 있다.
    - 특히 chatbot같이 비슷한 질문을 하는 경우 유용하다.
- 저장 방식은 `메모리`, `sqlite` 등 다양한 방식을 지원한다.
  
    ```python
    set_llm_cache(Cache객체)
    ```